In [1]:
with open("input_12.txt", "r") as fh:
    grid = [[c for c in r.strip()] for r in fh.readlines()]

In [2]:
def search_adj_plots(letter, coord):
    # Determine adjacent plots with same letter, and give permiter
    r, c = coord
    adj_coords = set()
    for r_adj, c_adj in [(r-1, c), (r+1, c), (r, c-1), (r, c+1)]:
        if 0 <= r_adj < len(grid) and 0 <= c_adj < len(grid[0]) and grid[r_adj][c_adj] == letter:
            # Same region
            adj_coords.add((r_adj, c_adj))
    plot_perimeter = 4 - len(adj_coords)
    return adj_coords, plot_perimeter

def search_region(letter, coord):
    # Search entire region with letter and start_coord
    search_coords = set([coord])
    region_coords = set()
    region_perimeter = 0
    while len(search_coords):
        c = search_coords.pop()
        if c in region_coords:
            continue
        adj_coords, plot_perimeter = search_adj_plots(letter, c)
        search_coords = search_coords.union(adj_coords)
        region_perimeter += plot_perimeter
        region_coords.add(c)
    return region_coords, region_perimeter*len(region_coords)

In [3]:
# 12a
seen_coords = set()
total_cost = 0
for ir, r in enumerate(grid):
    for ic, c in enumerate(r):
        if (ir, ic) not in seen_coords:
            region_coords, region_cost = search_region(c, (ir, ic))
            seen_coords = seen_coords.union(region_coords)
            total_cost += region_cost
print("12a:", total_cost)

12a: 1402544


In [4]:
# 12b
def get_sides(region_coords):
    hor_sides = {} # k=height (on top is r, on bottom is r+1), v=column (plot) number
    ver_sides = {} # k=lateral position (left is c, right is c+1), v=row (plot) number
    for r, c in region_coords:
        letter = grid[r][c]
    
        # Horizontal
        if r-1 < 0 or (r-1 >= 0 and grid[r-1][c] != letter):
            if r not in hor_sides:
                hor_sides[r] = set()
            hor_sides[r].add(-c) # Negative c is only added to distinguish sides next to each other that enclose different plots
        if r+1 >= len(grid) or (r+1 < len(grid) and grid[r+1][c] != letter):
            if r+1 not in hor_sides:
                hor_sides[r+1] = set()
            hor_sides[r+1].add(c)
        # Vertical
        if c-1 < 0 or (c-1 >= 0 and grid[r][c-1] != letter):
            if c not in ver_sides:
                ver_sides[c] = set()
            ver_sides[c].add(-r)
        if c+1 >= len(grid[0]) or (c+1 < len(grid[0]) and grid[r][c+1] != letter):
            if c+1 not in ver_sides:
                ver_sides[c+1] = set()
            ver_sides[c+1].add(r)
    
    # Each set in hor_sides and ver_sides now represents one or multiple sides on a certain line; no need to distinguish
    sides = []
    for hs in hor_sides.values():
        sides.append(hs)
    for vs in ver_sides.values():
        sides.append(vs)
    
    return sides

seen_coords = set()
total_cost = 0
for ir, r in enumerate(grid):
    for ic, c in enumerate(r):
        if (ir, ic) not in seen_coords:
            region_coords, region_cost = search_region(c, (ir, ic))
            seen_coords = seen_coords.union(region_coords)
            # Extract all sides of the region
            sides = get_sides(region_coords)
            num_sides = 0
            # A sequence of numbers is 1 side if they increment by 1, for any larger increment, it indicates an added side.
            for s in sides:
                s = sorted(s)
                num_sides += 1 # always minimum of 1 side
                for i in range(len(s)-1):
                    if s[i+1] - s[i] > 1:
                        num_sides += 1
            total_cost += len(region_coords) * num_sides
print("12b:", total_cost)

12b: 862486
